In [ ]:
from lifelines import WeibullAFTFitter
from pyspark.sql.session import SparkSession, SparkConf
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)
import pandas as pd
import seaborn as sns
import numpy as np
from lifelines.utils import k_fold_cross_validation

In [ ]:
df = pd.read_parquet('dados_modelo.parquet')

In [ ]:
df = df.dropna()

In [ ]:
cnpj = df['cnpj']
df.drop(columns=['cnpj'])

In [ ]:
df = pd.get_dummies(df,drop_first=True)

In [ ]:
df = df[df['idade_anos'] > 0]

In [ ]:
aft = WeibullAFTFitter(penalizer=0.01)

In [ ]:
df = df.dropna()

In [ ]:
df.columns

In [ ]:
df[['latitude', 'longitude']]

In [ ]:
df['latitude'] = df['latitude'] + 21.558548
df['longitude'] = df['longitude'] + 45.4394151

In [ ]:
df.drop(columns=['grande_area_ELETRICIDADE E GAS'], inplace=True)

In [ ]:
aft.fit(df, duration_col='idade_anos', event_col='fechada', ancillary=True)

In [ ]:
aft.print_summary()

In [ ]:
print(aft.median_survival_time_)
print(aft.mean_survival_time_)

In [ ]:
aft.plot()

In [ ]:
df['latitude'].min()

In [ ]:
aft.plot_partial_effects_on_outcome('latitude', values=np.arange(-2,2,0.5),cmap='coolwarm')

In [ ]:
abertas = df[df['fechada'] == 0]

In [ ]:
pred = aft.predict_median(abertas)

In [ ]:
pred.max()

In [ ]:
scores = k_fold_cross_validation(aft, df, duration_col='idade_anos', event_col='fechada', k=3, scoring_method="concordance_index")
print(scores)